In [33]:
import pandas as pd
import weaviate
import weaviate.classes as wvc

blockchain_tx_log_DF = pd.DataFrame(columns=['transaction_log'])

# Loading the kiosk data into a DataFrame
blockchain_tx_log_DF = pd.read_json('data/digest__metadata.json', lines=True)

display(blockchain_tx_log_DF)

,response,digest
0,{'edges': []},EyKYy7fv3XyDMR2R4wHpGbDWfBtboeBWmHxybX1TxsE4
1,{'edges': [{'node': {'timestamp': '2024-05-31T...,EQMPKtdqee4dAKvyaSH7JpU95o622peYinhydNw72Lbe
2,{'edges': [{'node': {'timestamp': '2024-05-31T...,3CwH6TVFVAm7AhFo7fSaoooy7TQpRE638ZCNvx8cufiW
3,{'edges': []},BjP6oFDfXqHuYc1mv8FFBm5Wp9Xt58FqX66JNc1dyXuD
4,{'edges': []},DHi6KTA5pz1Hn6R8pTKgwvavaFnSdR6zszocgUxgJJM6
...,...,...
995,{'edges': [{'node': {'timestamp': '2024-05-28T...,AxCHRU9VG5BtUF7oyaxVLCx5umGjjd6bHZpGCUPaH713
996,{'edges': [{'node': {'timestamp': '2024-05-28T...,85aLyYXayszxcD8ocZrnRyVVuEH6ZpFmReXWQ6ou3pk3
997,{'edges': []},ESTdAWVYuaUGqyw1zPK2wXSNgCjsRXnMCLpEMxsmz1cL
998,{'edges': []},vi3125pxJXGT9uGr1cXQTuNit6csLnAiHEwoaaow342


In [ ]:
WEAVIATE_CLUSTER_URL = 'WEAVIATE_CLUSTER_URL'
WEAVIATE_API_KEY = 'WEAVIATE_API_KEY'
OPENAI_API_KEY = 'OPENAI_API_KEY'

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_CLUSTER_URL,
    auth_credentials=weaviate.AuthApiKey(api_key=WEAVIATE_API_KEY),
    headers={"X-OpenAI-Api-Key": OPENAI_API_KEY})

In [35]:
collection_name = "sui_blockchain_game_metadata"


In [ ]:
#If collection does not exist
collection = client.collections.create(
    name = collection_name,
    vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_openai(),  # If set to "none" you must always provide vectors yourself. Could be any other "text2vec-*" also.
    generative_config=wvc.config.Configure.Generative.openai()  # Ensure the `generative-openai` module is used for generative queries
)

In [36]:
# Upload each transaction response metadata
metadata_objs = list()

try:
    for row in range(len(blockchain_tx_log_DF)):
        metadata_objs.append({"platform": str(blockchain_tx_log_DF['response'][row])})
except Exception as e:
    print(f"Exception: {e}")

metadata = client.collections.get(collection_name)
metadata.data.insert_many(metadata_objs)

client.close()  # Close client gracefully

In [47]:
metadata_objs

[{'platform': "{'edges': []}"},
 {'platform': "{'edges': [{'node': {'timestamp': '2024-05-31T00:29:38.476Z', 'data': {'Struct': [{'name': 'game_id', 'value': {'ID': [194, 106, 72, 202, 38, 111, 225, 56, 182, 57, 241, 157, 56, 211, 144, 21, 135, 6, 10, 31, 31, 26, 122, 159, 60, 72, 50, 215, 87, 230, 123, 149]}}, {'name': 'current_player_hand_sum', 'value': {'Number': '22'}}, {'name': 'player_cards', 'value': {'Vector': [{'Number': '18'}, {'Number': '11'}, {'Number': '44'}]}}]}, 'json': {'game_id': '0xc26a48ca266fe138b639f19d38d3901587060a1f1f1a7a9f3c4832d757e67b95', 'current_player_hand_sum': 22, 'player_cards': [18, 11, 44]}, 'type': {'repr': '0x8f4ba9c068cc90c1140d7310ca81b8db1f43ec38cfb252ec227ac307d8829c8d::single_player_blackjack::HitDoneEvent', 'signature': {'datatype': {'package': '0x8f4ba9c068cc90c1140d7310ca81b8db1f43ec38cfb252ec227ac307d8829c8d', 'module': 'single_player_blackjack', 'type': 'HitDoneEvent', 'typeParameters': []}}, 'layout': {'struct': {'type': '0x8f4ba9c068cc90

In [ ]:
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_CLUSTER_URL,
    auth_credentials=weaviate.AuthApiKey(api_key=WEAVIATE_API_KEY),
    headers={"X-OpenAI-Api-Key": OPENAI_API_KEY})

In [49]:
metadata = client.collections.get("Sui_blockchain_game_metadata")

response = metadata.generate.near_text(
    query="Game ID",
    limit=2,
    single_prompt="How many unqiue game IDs can be found in the nested JSON of {platform}"
)

print(response.objects[0].generated)  # Inspect the generated text

There are 32 unique game IDs in the nested JSON data provided.


In [51]:
metadata = client.collections.get("Sui_blockchain_game_metadata")

response = metadata.generate.near_text(
    query="cards",
    limit=2,
    single_prompt="What is the average winning card amount in {platform} based off the field current_player_hand_sum"
)

print(response.objects[0].generated)  # Inspect the generated text

To calculate the average winning card amount based on the field current_player_hand_sum, we need to sum up all the values of current_player_hand_sum and then divide by the total number of values.

In the provided data, the current_player_hand_sum values are: 16

Therefore, the average winning card amount based on the field current_player_hand_sum is:

(16) / 1 = 16

So, the average winning card amount is 16.


In [55]:
metadata = client.collections.get("Sui_blockchain_game_metadata")

response = metadata.generate.near_text(
    query="cards",
    limit=2,
    single_prompt="What is the most common card drawn from the raw logs found in {platform}"
)

print(response.objects[0].generated)  # Inspect the generated text

The most common card drawn from the raw logs is the number 42.
